In [2]:
import numpy as np, pandas as pd, json, glob, os, chromadb, openai
# from sentence_transformers import SentenceTransformer
from openai import OpenAI
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
from tqdm.auto import tqdm
import shutil
from chromadb import Documents, EmbeddingFunction, Embeddings


/home/ethan/anaconda3/envs/pl_llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
key = open('../../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

base_ = open('../../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

In [4]:
class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model: str = "bge-m3"):
        self.model = model
        self.embedding_model = OllamaEmbeddings(model=self.model,
                                                # base_url = base_url,
                                                )

    def __call__(self, input: Documents) -> Embeddings:
        # input은 List[str] 형태로 들어옴
        if isinstance(input, str):
            input = [input]

        embeddings = [self.embedding_model.embed_query(doc) for doc in input]
        return np.array(embeddings).tolist()  # ChromaDB는 List[List[float]] 기대
embedding_fn = MyEmbeddingFunction(model="bge-m3")


In [5]:
# embedding_fn

In [5]:
db_path = "../data/chroma_db"
if os.path.exists(db_path):
    shutil.rmtree(db_path)
    print("기존 ChromaDB 삭제 완료")

기존 ChromaDB 삭제 완료


In [6]:
# from chromadb.utils import embedding_functions

# bge_embed = embedding_functions.SentenceTransformerEmbeddingFunction(
#     model_name="BAAI/bge-m3"
# )

In [6]:
# ChromaDB 클라이언트
db_path = "../data/chroma_db"
client = chromadb.PersistentClient(path =db_path)

#collection 삭제
# client.delete_collection('medical_qa')

# Collection 생성
collection = client.get_or_create_collection('medical_qa',
                                             embedding_function=embedding_fn
                                             )


In [15]:
'../../../medical_data/02.라벨링데이터/'

'../../../medical_data/02.라벨링데이터/'

In [16]:
# base_url = 'http://211.170.172.234/'

# 삽입 기능

In [17]:
path = '../../../medical_data/02.라벨링데이터/'
for domain in os.listdir(path):
    print(domain.split('/')[-1])
    for file in tqdm(os.listdir(os.path.join(path, domain))):
        file_path = os.path.join(path, domain, file)
        with open(file_path, 'r', encoding='utf-8-sig') as f:
            json_file = f.read()
            data = json.loads(json_file)

        # print(data)
        question = data.get("question" , "").strip()
        # print(question)
    # break
        # embedding = embedding_fn(question)
        # embedding = embedding.astype('float64').tolist()
        # print(embedding)
        # break
    # break
        # embedding = embedding.astype(float).tolist()
        # # print(embedding)
        collection.add(
            ids = [file],
            documents=[question],
            # embeddings = embedding,
            metadatas = [{
                "answer" : data['answer'],
                "qa_id" : data["qa_id"],
                "domain" : data["domain"],
                "q_type" : data["q_type"],
            }],
        )     
        # collection.update(
        #     ids = [file],
        #     # documents=[question],
        #     embeddings = [embedding],
            
        # )   

        # break
    # break
        
    

TL_내과


100%|██████████| 10299/10299 [1:17:16<00:00,  2.22it/s]


TL_응급의학과


100%|██████████| 649/649 [04:20<00:00,  2.50it/s]


TL_산부인과


100%|██████████| 1991/1991 [14:27<00:00,  2.30it/s]


TL_소아청소년과


100%|██████████| 2421/2421 [18:36<00:00,  2.17it/s]


In [15]:
collection.query(
    query_texts=["신생아 중환자실에서 기계환기를 받고 있는"],
    n_results=3,
    where={"domain":15}
    
    
)

{'ids': [['필수_62.json', '필수_21009.json', '필수_620175.json']],
 'embeddings': None,
 'documents': [["신생아 중환자실에서 기계환기를 받고 있는 3주 된 미숙아가 폐 분비물 제거를 위해 치료를 받고 있다. 다음 중 폐 분비물 제거를 위한 흉부 물리치료(CPT) 기법으로 권장되지 않는 것은 무엇인가?  \n1) 자세배액, 타진, 진동(Postural Drainage, Percussion, and Vibration) \n2) 흡인(Suction) 단독  \n3) 발꿈치 접촉 타진(Contact Heel Percussion) \n4) '컵핑' 타진(Cupping Percussion) \n5) 폐 짜기 기법(Lung Squeezing Technique)",
   '8세 남아가 경련을 주소로 내원하였다. 10일 전 고열, 인두통, 구토가 있었고, 하루 뒤 목, 가슴 윗부분 및 허벅지 부위에 발진이 나타났으나 해열제 치료 후 5일 만에 소실되었다. 내원 전날 소변량 감소, 콜라색 소변, 안면 부종이 발생하였다. 이 환아에서 가장 시급히 시행해야 할 처치는 무엇인가?  \n1) 혈압을 측정한다.  \n2) 스테로이드를 투여한다.  \n3) 알부민을 정맥 주사한다.  \n4) 항생제를 투여한다.  \n5) 생리식염수를 정맥 주사한다.',
   '신생아 선별검사를 통해 조기에 발견하지 못하면 지능 저하와 같은 돌이킬 수 없는 손상이 발생할 수 있는 내분비 질환은 무엇인가?']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'answer': '5) 폐 짜기 기법(Lung Squeezing Technique)',
    'q_type': 1,
    'qa_id': 62,
    'domain': 15},
   {'qa_id': 21009, 'q_type': 1, 'dom

In [ ]:
collection.query(
    query_texts=["신생아 중환자실에서 기계환기를 받고 있는"],
    n_results=3,
    where={"domain":15}
    
    
)

In [12]:
all_doc = collection.get()